## Environment & Imports

In [ ]:
# -*- coding: utf-8 -*-
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib
!pip install XlsxWriter
! pip install ipdb -q

import ipdb
import xlsxwriter
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as datetime
import numpy as np

## Configuration & Parameters

In [ ]:
#import talib
#!pip install yfinance --upgrade --no-cache-dir
#import requests
#import yfinance as yf
import warnings
from scipy.stats import zscore

from google.colab import files
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics

from google.colab import drive
drive.mount('/content/drive')

def training(mem, drpout, r_drpout, i_shp1, i_shp2, out):
        model = Sequential()
        model.add(LSTM(mem, dropout=drpout, recurrent_dropout=r_drpout,input_shape=(i_shp1, i_shp2), activation='tanh'))
        model.add(Dense(out))
        model.compile(loss='mae', optimizer='Adam', metrics=['accuracy'])
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, verbose=0, mode='auto')
        checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=1, save_best_only=True) # save best model
        model.fit(train_X,train_y,validation_data=(test_X,test_y),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
        model.load_weights('best_weights.hdf5') # load weights from best model
        return model

# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.show()

path = "/content/drive/My Drive/Danismanlik/Horizon/AI/Apple/dat_March_20a.csv"

## Data Loading

In [ ]:
df = pd.read_csv(path, index_col=1)

# NaN data checking
n_nan_dat = pd.isna(df['temp_min']).sum() 
print(n_nan_dat)
# 158 data points are missing in each label (temp_min, temp_max, temp_avg, humig_pctg_min, humid_pctg_max, humid_pctg_avg
df['temp_min'].interpolate(inplace=True)
df['temp_max'].interpolate(inplace=True)
df['temp_avg'].interpolate(inplace=True)
df['humig_pctg_min'].interpolate(inplace=True)
df['humid_pctg_max'].interpolate(inplace=True)
df['humid_pctg_avg'].interpolate(inplace=True)

## Preprocessing

In [ ]:
values = df.values
# specify columns to plot
groups = [6, 11, 14, 15, 16]
i = 1
# plot each column
plt.figure()
for group in groups:
	plt.subplot(len(groups), 1, i)
	plt.plot(values[:, group])
	plt.title(df.columns[group], y=0.5, loc='right')
	i += 1
plt.show()

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
		# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	# https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
	return agg

#n_df = df[['calculated_egg', 'int_dd', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
#n_df = df[['calculated_egg', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
n_df = df[['calculated_egg', 'temp_min', 'temp_max']]
values = n_df.values
# ensure all data is float
values = values.astype('float32')

#datset=np.array(n_df)

# first we need to normalize our dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

look_back = 7*24
# look_back = 1

reframed = series_to_supervised(scaled, look_back, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[reframed.shape[1]-n_df.shape[1]+1:reframed.shape[1]], axis=1, inplace=True)
print(reframed.head())
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_hours = 1 * 9 * 30 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

model= training(100, 0, 0, train_X.shape[1], train_X.shape[2], 1)

pred = model.predict(test_X)

# Plot the chart
chart_regression(pred.flatten(),test_y, sort=False)

# grab some data
#ts, tmp_avg, tmp_max, tmp_min, hm_avg, hm_max, hm_min = np.double(df.loc[:,('unix_time_stamp')]),np.double(df.loc[:,('temp_avg')]), np.double(df.loc[:,('temp_max')]), np.double(df.loc[:,('temp_min')]), np.double(df.loc[:,('humid_pctg_avg')]), np.double(df.loc[:,('humid_pctg_max')]), np.double(df.loc[:,('humig_pctg_min')])

datset = np.array(n_df)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(n_df)
dtset = scaler.transform(datset)
tavg =scaler.inverse_transform(dtset)[:,1]      # check orginal values of temp_avg

n_df.count()

Jan17['temp_avg'].plot()

Jan17=df.loc[(df['year_'] == 2017) & (df['month_'] == 1)]

testY.shape

df.count()

df

## Training

In [ ]:
        model.fit(train_X,train_y,validation_data=(test_X,test_y),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
        model.load_weights('best_weights.hdf5') # load weights from best model
        return model

# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.show()

path = "/content/drive/My Drive/Danismanlik/Horizon/AI/Apple/dat_March_20a.csv"
df = pd.read_csv(path, index_col=1)

# NaN data checking
n_nan_dat = pd.isna(df['temp_min']).sum() 
print(n_nan_dat)
# 158 data points are missing in each label (temp_min, temp_max, temp_avg, humig_pctg_min, humid_pctg_max, humid_pctg_avg
df['temp_min'].interpolate(inplace=True)
df['temp_max'].interpolate(inplace=True)
df['temp_avg'].interpolate(inplace=True)
df['humig_pctg_min'].interpolate(inplace=True)
df['humid_pctg_max'].interpolate(inplace=True)
df['humid_pctg_avg'].interpolate(inplace=True)
values = df.values
# specify columns to plot
groups = [6, 11, 14, 15, 16]
i = 1
# plot each column
plt.figure()
for group in groups:
	plt.subplot(len(groups), 1, i)
	plt.plot(values[:, group])
	plt.title(df.columns[group], y=0.5, loc='right')
	i += 1
plt.show()

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
		# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	# https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
	return agg

#n_df = df[['calculated_egg', 'int_dd', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
#n_df = df[['calculated_egg', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
n_df = df[['calculated_egg', 'temp_min', 'temp_max']]
values = n_df.values
# ensure all data is float
values = values.astype('float32')

#datset=np.array(n_df)

# first we need to normalize our dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

look_back = 7*24
# look_back = 1

reframed = series_to_supervised(scaled, look_back, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[reframed.shape[1]-n_df.shape[1]+1:reframed.shape[1]], axis=1, inplace=True)
print(reframed.head())
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_hours = 1 * 9 * 30 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

model= training(100, 0, 0, train_X.shape[1], train_X.shape[2], 1)

pred = model.predict(test_X)

# Plot the chart
chart_regression(pred.flatten(),test_y, sort=False)

# grab some data
#ts, tmp_avg, tmp_max, tmp_min, hm_avg, hm_max, hm_min = np.double(df.loc[:,('unix_time_stamp')]),np.double(df.loc[:,('temp_avg')]), np.double(df.loc[:,('temp_max')]), np.double(df.loc[:,('temp_min')]), np.double(df.loc[:,('humid_pctg_avg')]), np.double(df.loc[:,('humid_pctg_max')]), np.double(df.loc[:,('humig_pctg_min')])

datset = np.array(n_df)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(n_df)
dtset = scaler.transform(datset)
tavg =scaler.inverse_transform(dtset)[:,1]      # check orginal values of temp_avg

n_df.count()

Jan17['temp_avg'].plot()

Jan17=df.loc[(df['year_'] == 2017) & (df['month_'] == 1)]

testY.shape

df.count()

df

## Prediction & Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as datetime
import numpy as np
#import talib
#!pip install yfinance --upgrade --no-cache-dir
#import requests
#import yfinance as yf
import warnings
from scipy.stats import zscore

from google.colab import files
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics

from google.colab import drive
drive.mount('/content/drive')

def training(mem, drpout, r_drpout, i_shp1, i_shp2, out):
        model = Sequential()
        model.add(LSTM(mem, dropout=drpout, recurrent_dropout=r_drpout,input_shape=(i_shp1, i_shp2), activation='tanh'))
        model.add(Dense(out))
        model.compile(loss='mae', optimizer='Adam', metrics=['accuracy'])
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, verbose=0, mode='auto')
        checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=1, save_best_only=True) # save best model
        model.fit(train_X,train_y,validation_data=(test_X,test_y),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
        model.load_weights('best_weights.hdf5') # load weights from best model
        return model

# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.show()

path = "/content/drive/My Drive/Danismanlik/Horizon/AI/Apple/dat_March_20a.csv"
df = pd.read_csv(path, index_col=1)

# NaN data checking
n_nan_dat = pd.isna(df['temp_min']).sum() 
print(n_nan_dat)
# 158 data points are missing in each label (temp_min, temp_max, temp_avg, humig_pctg_min, humid_pctg_max, humid_pctg_avg
df['temp_min'].interpolate(inplace=True)
df['temp_max'].interpolate(inplace=True)
df['temp_avg'].interpolate(inplace=True)
df['humig_pctg_min'].interpolate(inplace=True)
df['humid_pctg_max'].interpolate(inplace=True)
df['humid_pctg_avg'].interpolate(inplace=True)
values = df.values
# specify columns to plot
groups = [6, 11, 14, 15, 16]
i = 1
# plot each column
plt.figure()
for group in groups:
	plt.subplot(len(groups), 1, i)
	plt.plot(values[:, group])
	plt.title(df.columns[group], y=0.5, loc='right')
	i += 1
plt.show()

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
		# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	# https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
	return agg

#n_df = df[['calculated_egg', 'int_dd', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
#n_df = df[['calculated_egg', 'temp_min', 'temp_max', 'humig_pctg_min', 'humid_pctg_max']]
n_df = df[['calculated_egg', 'temp_min', 'temp_max']]
values = n_df.values
# ensure all data is float
values = values.astype('float32')

#datset=np.array(n_df)

# first we need to normalize our dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)

look_back = 7*24
# look_back = 1

reframed = series_to_supervised(scaled, look_back, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[reframed.shape[1]-n_df.shape[1]+1:reframed.shape[1]], axis=1, inplace=True)
print(reframed.head())
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_hours = 1 * 9 * 30 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

model= training(100, 0, 0, train_X.shape[1], train_X.shape[2], 1)

pred = model.predict(test_X)

# Plot the chart
chart_regression(pred.flatten(),test_y, sort=False)

# grab some data
#ts, tmp_avg, tmp_max, tmp_min, hm_avg, hm_max, hm_min = np.double(df.loc[:,('unix_time_stamp')]),np.double(df.loc[:,('temp_avg')]), np.double(df.loc[:,('temp_max')]), np.double(df.loc[:,('temp_min')]), np.double(df.loc[:,('humid_pctg_avg')]), np.double(df.loc[:,('humid_pctg_max')]), np.double(df.loc[:,('humig_pctg_min')])

datset = np.array(n_df)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(n_df)
dtset = scaler.transform(datset)
tavg =scaler.inverse_transform(dtset)[:,1]      # check orginal values of temp_avg

n_df.count()

Jan17['temp_avg'].plot()

Jan17=df.loc[(df['year_'] == 2017) & (df['month_'] == 1)]

testY.shape

df.count()

df